# Introduction to multislice simulations with abTEM

This tutorial is a short introduction to image simulation with abTEM. The tutorial covers some basic principles and presents examples for CBED, HRTEM and (4D-)STEM simulations. For more in depth information, and information on other simulation modes, see the following resources:

* [The abTEM walkthrough](https://abtem.readthedocs.io/en/latest/walkthrough/introduction.html)
* [Quickstart examples](https://github.com/jacobjma/abTEM/tree/master/examples)
* [Examples repository](https://github.com/jacobjma/abTEM/tree/master/examples)

### Contents:

1. <a href='#import'> Import atomic model
2. <a href='#potentials'> Creating Potentials with the IAM
3. <a href='#waves'> The Waves object
4. <a href='#probes'> Creating Probe wave functions
5. <a href='#multislice'> Multislice simulation with Probe
6. <a href='#scan'> Scanned multislice simulation
7. <a href='#hrtem'> HRTEM simulation
8. <a href='#frozen_phonons'> The frozen phonon model
9. <a href='#prism'> Large STEM simulation with PRISM

### Author:
* 20/05/2023 Jacob Madsen - For the HyperSpy workshop at ePSIC 2023

In [ ]:
%matplotlib qt5

import ase
import numpy as np
import matplotlib.pyplot as plt

import abtem as ab

np.set_printoptions(edgeitems=1)

print("Tested with abTEM v1.0.0beta32. Your current version:", ab.__version__)

## Import atomic model <a id='import'></a>

To start running image simulations, we need an atomic model. Creating an atomic model is covered in "atomic_models_with_ase.ipynb", if you do not have the file "sto_lto.cif", please run that notebook first.

In [ ]:
atoms = ase.io.read("sto_lto.cif")

fig, (ax1, ax2) = plt.subplots(1, 2)
ab.show_atoms(atoms, ax=ax1, plane="xy", title="Beam view")
ab.show_atoms(atoms, ax=ax2, plane="xz", title="Side view")

## Creating `Potential`'s with the independent atom model <a id='potentials'></a>
We use the indepedent atom model (IAM) to create the electrostatic potential of the sample, in this model the potential is a superposition of parametrizations of single atomic potentials. 

To define a `Potential`, we need to provide an ASE atoms object, a sampling rate (or pixel size) in $x$ and $y$, and a slice thickness in the $z$-direction (the propagation direction). 

The multislice algorithm is only accurate in the limit of thin slices, however, thin slices also increases computational cost. A sensible value for the slice thickness is typically between $0.5 \ \mathrm{Å}$ and $2 \ \mathrm{Å}$.

In [ ]:
potential = ab.Potential(atoms, sampling=0.05, slice_thickness=2)

The `.build` method is available for many simulation objects. This method will convert a simulation object into a static array-based object.

In [ ]:
potential_array = potential.build()

In [ ]:
potential_array.array

We can show the projected potential using the `.show` method.

In [ ]:
potential_array.project().show(cmap="viridis");

abTEM has some direct integration with HyperSpy. Hence, some objects can be converted directly to HyperSpy signals.

In [ ]:
potential_signal = potential_array.to_hyperspy()

In [ ]:
potential_signal.axes_manager

We can use the `.plot` method introduced earlier in the workshop.

In [ ]:
potential_signal.plot(navigator="slider", cmap="viridis")

## Creating `Probe` wave functions <a id='probes'></a>

The multislice algorithm works by propagating the $xy$ part of the wave function through the electrostatic potential along the $z$-axis. In STEM, the wave function is a focused beam of electrons. The convention used in abTEM is a probe defined by

$$
    \phi(\mathbf{k}, \mathbf{r}_0) = A(k) \exp(-i \chi(\mathbf{k})) \exp(-i 2 \pi \mathbf{k} \cdot \mathbf{r}_p) \quad ,
$$

where $\mathbf{k} = (k_x, k_y)$ is the spatial frequency, $A(k)$ is the condenser aperture function and $\chi(\mathbf{k})$ is the phase error and $\mathbf{r}_p = (x_p, y_p)$ is the probe position.

If the microscope is well aligned then off-axis aberrations are small and the phase error is dominated by defocus and spherical aberration

$$
    \chi(k) \approx \frac{2\pi}{\lambda}\left( \frac{\lambda^2 k^2}{2} \Delta f + \frac{\lambda^4 k^4}{4} C_s \right) \quad ,
$$

where $\Delta f$ is the defocus and $C_s$ is the third order spherical aberration. See our [walkthrough section](https://abtem.readthedocs.io/en/latest/walkthrough/05_contrast_transfer_function.html) for more information.

We create a probe with an energy of $200 \ \mathrm{keV}$, a convergence semiangle of $20 \ \mathrm{mrad}$, and a defocus of $50 \ \mathrm{Å}$.

__Note__: Positive defocus is equivalent to backward free-space propagation, i.e. a probe with positive defocus is "in focus" inside the sample.

In [ ]:
probe = ab.Probe(energy=200e3, defocus=50, semiangle_cutoff=20)

We match our probe to our potential.

In [ ]:
probe.grid.match(potential)

We may want to `.build` the probe to obtain an array representation.

In [ ]:
probe_waves = probe.build()

In [ ]:
probe_waves.array

The wave function intensity can be shown in real or reciprocal space using the `.intensity` or `.diffraction_pattern` method and showing the resulting `Measurement`.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
probe_waves.intensity().show(ax=ax1, cmap="viridis")
probe_waves.diffraction_pattern().show(ax=ax2, cmap="viridis");

## Multislice simulation with a `Probe` (CBED) <a id='multislice'></a>
We use the multislice algorithm to propagate the probe through the potential. We can choose where to place the probe by setting the `positions`.

In [ ]:
position = (6, 6)

exit_wave = probe.build(positions=position).multislice(potential)

We show the `.intensity` and `.diffraction_pattern` as above.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
exit_wave.intensity().show(ax=ax1, cmap="inferno")
exit_wave.diffraction_pattern(max_angle=None).show(cmap="inferno", power=0.5, ax=ax2);

The real-space sampling determines the maximum simulated scattering angle. The sampling defines the maximum spatial frequency $k_{max}$ via the formula:

$$ k_{max} = \frac{1}{2d} \quad , $$

where $d$ is the real-space sampling distance. To counteract aliasing artifacts due to the periodicity assumption of a discrete Fourier transform, abTEM supresses spatial frequencies above 2 / 3 of the maximum scattering angle, further reducing the maximum effective scattering angle by a factor of 2 / 3. Hence the maximum scattering angle $\alpha_{max}$ is given by:

$$ \alpha_{max} = \frac{2}{3}\frac{\lambda}{2p} \quad , $$

where $\lambda$ is the relativistic electron wavelength. 

## Scanned multislice simulation (HAADF and 4D-STEM) <a id='scan'></a>

Scanning imaging modes such as STEM works by rastering an electron probe across a sample pixel by pixel and recording the scattering signal. 

We create a grid scan and set the sampling (probe step size) to the Nyquist sampling of the probe. 

In [ ]:
scan = ab.GridScan(
    start=(0, 0),
    end=(potential.extent[0] / 2, potential.extent[1]),
    sampling=probe.ctf.nyquist_sampling,
)

__Note__: The scan sampling should not be confused with the wave function sampling. The scan sampling in any integrated STEM imaging mode should generally be set to the Nyquist sampling rate. The image can be interpolated to the typically much higher experimental sampling rate.
</div>

In abTEM the exit waves are "detected" using a detector object. There are several different types of detectors, the most basic, the `AnnularDetector`, may be used for bright-field, medium- or high-angle annular dark-field microscopy. Depending on the integration region.

The integration region is given by an inner and outer radius in mrad; below we create three different types of detectors. We show the integration region of the HAADF detector.

The `PixelatedDetector` will detect a full diffraction pattern for each probe position.

In [ ]:
haadf_detector = ab.AnnularDetector(inner=80, outer=200)
pixelated_detector = ab.PixelatedDetector()

haadf_detector.show(probe);

The scanned multislice simulations are started as below, a progress bar is shown to indicate how long time the simulation will take. It takes about 60s on my 2018 Macbook.

In [ ]:
scanned_measurements = probe.scan(
    scan=scan,
    detectors=[haadf_detector, pixelated_detector],
    potential=potential,
    pbar=True,
)

haadf, pixelated = scanned_measurements

We show the measurements as earlier.

In [ ]:
haadf.show(title="STEM-ADF [80, 200] mrad");

We can use hyperspy to explore the 4D dataset. We have to set `min_aspect=1` since HyperSpy plots square pixels, whereas our reciprocal space sampling is highly anisotropic. 

In [ ]:
pixelated.to_hyperspy().plot(min_aspect=1, cmap="viridis")

### Post-processing simulations

It is usually necessary to do some post-processing on the simulated images. In particular, we may want to resample the images. 

We usually also want to add a gaussian blur to simulate partial spatial coherence (i.e. source size). Partial temporal coherence (energy spread) is more costly to include and will not be covered here.

In [ ]:
interpolated_maadf = haadf.interpolate(0.1).gaussian_filter(0.35)

We create the final image by tiling it and adding Poisson noise.

In [ ]:
from abtem.noise import poisson_noise

noisy_maadf = poisson_noise(interpolated_maadf.tile((10, 4)), 5e4)
noisy_maadf.show(title="STEM-ADF [80, 200] mrad")

## HRTEM simulations with `PlaneWaves`'s <a id='hrtem'></a>

Running an HRTEM simulation is not very different from doing the CBED simulation. Instead of creating a `Probe`, we now create a `PlaneWave` with an energy of $200 \ \mathrm{keV}$.

In [ ]:
plane_wave = ab.PlaneWave(energy=200e3)

We run the mulislice algorithm.

In [ ]:
exit_wave = plane_wave.multislice(potential)

In [ ]:
exit_wave.show(cmap="viridis")

We obtained the exit wave, we now define the aberrations and aperture of the objective lens. We set the spherical abrration $C_s = -100 \ \mathrm{\mu m}$, the defocus is set to the Scherzer defocus ($\sim - 194 \ \mathrm{Å}$) and the aperture is convergence semiangle is set to match the point resolution ($\sim 20 \ \mathrm{mrad}$). We neglect partial coherence in this example.

In [ ]:
from abtem.transfer import scherzer_defocus

Cs = -100e-6 * 1e10
ctf = ab.CTF(Cs=Cs, energy=200e3)

ctf.defocus = scherzer_defocus(Cs, ctf.energy)
ctf.semiangle_cutoff = 20

ctf.show()

To obtain the image we apply the aberrations and aperture, then calculate the intensity. 

In [ ]:
image = exit_wave.apply_ctf(ctf).intensity()

We can also tile the result and apply Poisson noise. 

In [ ]:
noisy_image = poisson_noise(image.tile((5, 4)), 1e4)

noisy_image.show(title="HRTEM image")

## The frozen phonon model <a id='frozen_phonons'></a>
The atoms in any real material at a particular instance of time are not exactly located at their symmetrical lattice points due to thermal vibrations. In the Frozen phonon approximation, the effects of thermal vibrations are simulated by the _intensities_ averaged over several different configurations of atoms with different random offsets. 

To simulate frozen phonons the `Atoms` are wrapped with a `FrozenPhonons` object. To define a `FrozenPhonons` object we also need to provide the magnitude of the thermal vibrations for each atomic species.

Getting the right magnitude of thermal vibrations for a particular material, is not always trivial, here we just use the same reasonable value of $0.1 \ \mathrm{Å}$ for all atomic numbers. We set the number og random structures in the thermal ensemble to 8.

In [ ]:
frozen_phonons = ab.FrozenPhonons(atoms * (1, 1, 5), sigmas=0.1, num_configs=8)

We can draw a particular frozen phonon configuration by iterating.

In [ ]:
config = next(iter(frozen_phonons))

ab.show_atoms(config);

The potential can be created as above, we just provide the frozen phonons instead of the atoms. 

In [ ]:
frozen_phonon_potential = ab.Potential(frozen_phonons, sampling=0.05, slice_thickness=2)

The potential object can be used in the same way as above, here we do a CBED simulation, since the HAADF simulation is too time-consuming for a demo.

In [ ]:
wave = ab.PlaneWave(energy=200e3)

exit_wave = wave.multislice(potential=frozen_phonon_potential)

The output wave function is 3d, the first dimension is the frozen phonon ensemble dimension. 

In [ ]:
exit_wave.array.shape

To finalize the diffraction pattern we take the mean over the ensemble dimension.

In [ ]:
mean_diffraction_pattern = exit_wave.diffraction_pattern().mean(0)

We show the resulting diffraction pattern on a power scale.

In [ ]:
mean_diffraction_pattern.show(power=0.2, cmap="viridis");

__Note__: some imaging modes will average over frozen phonons by default to conserve memory.

## Large STEM simulation with PRISM <a id='prism'></a>
Multslice simulations of STEM images can be very slow because the scattering of the electron probe is calculated from scratch at each pixel of the image. An alternative is to use the [PRISM algorithm](https://prism-em.com). PRISM almost always provides a decent speed-up for images with many probe positions, and allows for huge speedups, at a modest cost to accuracy, by using Fourier interpolation.

We import the moderately large model of a nanoparticle on carbon that was constructed in "atomic_models_with_ase.ipynb".

In [ ]:
from ase.io import read

cluster = read("cluster_on_carbon.cfg")

print(
    "Number of atoms: {} \nCell: {:.2f} x {:.2f} x {:.2f}".format(
        len(cluster), *np.diag(cluster.cell)
    )
)

In abTEM, the PRISM algorithm can be used by simply exchanging the `Probe` for the `SMatrix`. However, there is one additional keyword the user has to know about, namely interpolation.

Increasing the interpolation decreases the number of plane waves necessary in the plane wave expansion of the probe, hence lowering both time and memory consumption. The necessary price is that the interpolation factor also decreases the Fourier space sampling rate, i.e. the pixels of the detected diffraction patterns becomes larger.

In [ ]:
S = ab.SMatrix(interpolation=6, energy=150e3, semiangle_cutoff=25)

We create potential, scan and a MAADF and pixelated detector as we have done before.

In [ ]:
cluster_potential = ab.Potential(cluster, gpts=512, slice_thickness=2)

cluster_scan = ab.GridScan(
    start=(0, 0), end=cluster_potential.extent, sampling=S.ctf.nyquist_sampling * 0.9
)

maadf_detector = ab.AnnularDetector(inner=60, outer=120)

The simulation is run as before. We lower the maximum number of plane waves propagated simulataneously to lower the memory footprint slightly.

In [ ]:
cluster_maadf = S.scan(
    potential=cluster_potential,
    scan=cluster_scan,
    detectors=maadf_detector,
    max_batch_expansion=10,
)

The simulation took around 30 seconds on my 8-core\@2.6 GHz laptop. The same simulation is estimated to take 4 hours on the same system.

Lastly, we postprocess and show the MAADF image, as we have done before.

In [ ]:
cluster_maadf.interpolate(0.1).gaussian_filter(0.3).show();